# Reward Distribution MODEL V4

This model handles Bond, BondPool and StakeManaer objects indivudually

In [178]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact
import math
import time

In [179]:
# Bond structure
class Bond:
    def __init__(self, amount, start_time, lock_duration, is_compound):
        self.amount = amount
        self.start_time = start_time
        self.lock_duration = lock_duration
        self.is_compound = is_compound
        self.earnings = 0
        self.claimable_reward = 0

    def weight(self):
        end_date = self.start_time + self.lock_duration * 7 * 24 * 60 * 60  # convert weeks to seconds
        remaining_duration = end_date - time.time()
        if remaining_duration < 0:
            remaining_duration = 1 / (24 * 60 * 60)  # 1 second
        return self.amount * math.log(remaining_duration + 1)

class BondPool:
    def __init__(self, name, stake_manager):
        self.name = name
        self.stake_manager = stake_manager
        self.bonds = []
        self.total_staked = 0
        self.total_earnings = 0
        self.total_claimable_rewards = 0
        self.total_weight = 0

    def add_bond(self, amount, start_time, lock_duration, is_compound):
        bond = Bond(amount, start_time, lock_duration, is_compound)
        self.bonds.append(bond)
        self.total_staked += amount
        bond_weight = bond.weight()
        self.total_weight += bond_weight
        self.stake_manager.update_abs_total_bond_weight(bond_weight)

    def update_reward(self, dao_rewards, abs_total_weight):
        for bond in self.bonds:
            bond_weight = bond.weight()
            bond_weight_normalized = bond_weight / abs_total_weight
            bond_reward = bond_weight_normalized * dao_rewards
            if bond.is_compound:
                bond.amount += bond_reward
                self.total_staked += bond_reward
                self.stake_manager.update_abs_total_staked(bond_reward, True)
            else:
                bond.claimable_reward += bond_reward
                self.total_claimable_rewards += bond_reward
            bond.earnings += bond_reward
            self.total_earnings += bond_reward

class StakeManager:
    def __init__(self):
        self.bond_pools = []
        self.abs_total_staked = 0
        self.abs_total_earnings = 0
        self.abs_total_claimable_rewards = 0
        self.abs_total_bond_weight = 0
        self.max_duration = 52  # 52 weeks
        self.max_start_time = 0

    def add_bond_pool(self, bond_pool):
        self.bond_pools.append(bond_pool)

    def update_abs_total_staked(self, amount, is_staking):
        if is_staking:
            self.abs_total_staked += amount
        else:
            self.abs_total_staked -= amount

    def update_abs_total_bond_weight(self, weight):
        self.abs_total_bond_weight += weight

    def get_abs_max_remain_duration(self):
        return self.max_duration * 7 * 24 * 60 * 60  # convert weeks to seconds

    def update_rewards(self, dao_rewards):
        for pool in self.bond_pools:
            pool.update_reward(dao_rewards, self.abs_total_bond_weight)

### Mock data 

In [180]:
# Create StakeManager and BondPools
stake_manager = StakeManager()

bond_pools_data = [
    [(1000, 4, False), (1500, 10, False)],
    [(1200, 20, False), (800, 52, True)],
    [(2000, 1, False)],
    [(500, 5, False), (700, 15, True), (900, 25, False)]
]

# Initialize BondPools
for i, bonds in enumerate(bond_pools_data):
    bond_pool = BondPool(f"Pool {i+1}", stake_manager)
    for amount, lock_duration, is_compound in bonds:
        bond_pool.add_bond(amount, time.time(), lock_duration, is_compound)
    stake_manager.add_bond_pool(bond_pool)


### Reward Distribution and Visualization

In [181]:
def visualize_rewards():
    data = []
    for pool in stake_manager.bond_pools:
        for bond in pool.bonds:
            data.append({
                "Pool": pool.name,
                "Amount": bond.amount,
                "Lock Duration": bond.lock_duration,
                "Is Compound": bond.is_compound,
                "Earnings": bond.earnings,
                "Claimable Reward": bond.claimable_reward
            })
    df = pd.DataFrame(data)
    return df

dao_rewards = 10000  # Example reward to distribute
stake_manager.update_rewards(dao_rewards)
df_rewards = visualize_rewards()

#### Test

In [182]:
def test_total_earnings(dao_rewards):
    total_earnings = sum(bond.earnings for pool in stake_manager.bond_pools for bond in pool.bonds)
    assert np.isclose(total_earnings, dao_rewards), f"Total earnings {total_earnings} does not match dao_rewards {dao_rewards}"
    print("All tests passed!")

# Run test
test_total_earnings(dao_rewards)

All tests passed!


### Interactive Visualization with Duration and dao_rewards Slider

In [183]:
def update_bond_duration(pool_name, bond_index, new_duration):
    pool = next(pool for pool in stake_manager.bond_pools if pool.name == pool_name)
    bond = pool.bonds[bond_index]
    bond.lock_duration = new_duration

def plot_rewards(pool_name, bond_index, new_duration, dao_rewards):
    update_bond_duration(pool_name, bond_index, new_duration)
    stake_manager.update_rewards(dao_rewards)
    
    pool = next(pool for pool in stake_manager.bond_pools if pool.name == pool_name)
    bond_data = {
        "Bond Index": range(len(pool.bonds)),
        "Amount": [bond.amount for bond in pool.bonds],
        "Lock Duration": [bond.lock_duration for bond in pool.bonds],
        "Is Compound": [bond.is_compound for bond in pool.bonds],
        "Earnings": [bond.earnings for bond in pool.bonds],
        "Claimable Reward": [bond.claimable_reward for bond in pool.bonds],
    }
    df_bonds = pd.DataFrame(bond_data)

    fig, ax = plt.subplots(figsize=(12, 6))
    df_bonds.plot(x="Bond Index", y=["Earnings", "Claimable Reward"], kind="bar", ax=ax)
    ax.set_title(f"Rewards Distribution in {pool_name} (dao_rewards={dao_rewards})")
    ax.set_ylabel("Amount")
    ax.set_xlabel("Bond Index")
    plt.show()

def interactive_plot(pool_name, bond_index, new_duration, dao_rewards):
    pool = next(pool for pool in stake_manager.bond_pools if pool.name == pool_name)
    bond_indices = range(len(pool.bonds))
    bond_durations = [bond.lock_duration for bond in pool.bonds]

    interact(plot_rewards, 
             pool_name=widgets.fixed(pool_name), 
             bond_index=widgets.IntSlider(min=0, max=len(pool.bonds)-1, step=1, value=bond_index), 
             new_duration=widgets.IntSlider(min=1, max=52, step=1, value=bond_durations[bond_index]),
             dao_rewards=widgets.IntSlider(min=1000, max=50000, step=1000, value=dao_rewards))

# Interactive plot for Pool 1 and bond index 0 with adjustable dao_rewards
interactive_plot("Pool 1", 0, 4, dao_rewards)

interactive(children=(IntSlider(value=0, description='bond_index', max=1), IntSlider(value=4, description='new…